In [17]:
import time
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from datetime import datetime
from kafka import KafkaProducer

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
# Change this to point to the folder where all the data remains
events = spark.read.csv("../2019-Dec.csv", header=True)

In [4]:
events.show(truncate=False)

+-----------------------+----------------+----------+-------------------+-------------+---------+-----+---------+------------------------------------+
|event_time             |event_type      |product_id|category_id        |category_code|brand    |price|user_id  |user_session                        |
+-----------------------+----------------+----------+-------------------+-------------+---------+-----+---------+------------------------------------+
|2019-12-01 00:00:00 UTC|remove_from_cart|5712790   |1487580005268456287|null         |f.o.x    |6.27 |576802932|51d85cb0-897f-48d2-918b-ad63965c12dc|
|2019-12-01 00:00:00 UTC|view            |5764655   |1487580005411062629|null         |cnd      |29.05|412120092|8adff31e-2051-4894-9758-224bfa8aec18|
|2019-12-01 00:00:02 UTC|cart            |4958      |1487580009471148064|null         |runail   |1.19 |494077766|c99a50e8-2fac-4c4d-89ec-41c05f114554|
|2019-12-01 00:00:05 UTC|view            |5848413   |1487580007675986893|null         |freedec

In [5]:
first_event = datetime.strptime(events.select("event_time").sort("event_time").first().event_time, "%Y-%m-%d %H:%M:%S %Z")

In [6]:
substract_date = f.udf(lambda event_time: (datetime.strptime(event_time, "%Y-%m-%d %H:%M:%S %Z") - first_event).seconds)

In [11]:
events_with_delta = events.withColumn("delta", substract_date(events.event_time))

In [25]:
current_delta = 0

In [26]:
producer = KafkaProducer(bootstrap_servers='localhost:9020')

while True:
    print("Current delta: {current_delta}".format(current_delta=current_delta))
    current_events = events_with_delta.filter(events_with_delta.delta == current_delta)
    current_timestamp = datetime.now()
    for event in current_events.collect():
        json_event = { "event_time": current_timestamp.strftime("%Y-%m-%d %H:%M:%S UTC"), 
                      "event_type": event.event_type, 
                      "product_id": event.product_id, 
                      "category_id": event.category_id, 
                      "brand": event.brand,
                      "price": event.price,
                      "user_id": event.user_id,
                      "user_session": event.user_session}
        # print(json_event)
        # Send event by kafka
        producer.send('ecommerce-events', json_event)
    time.sleep(1)
    current_delta += 1

Current delta: 0
{'event_time': '2023-10-05 10:55:12 UTC', 'event_type': 'remove_from_cart', 'product_id': '5712790', 'category_id': '1487580005268456287', 'brand': 'f.o.x', 'price': '6.27', 'user_id': '576802932', 'user_session': '51d85cb0-897f-48d2-918b-ad63965c12dc'}
{'event_time': '2023-10-05 10:55:12 UTC', 'event_type': 'view', 'product_id': '5764655', 'category_id': '1487580005411062629', 'brand': 'cnd', 'price': '29.05', 'user_id': '412120092', 'user_session': '8adff31e-2051-4894-9758-224bfa8aec18'}
{'event_time': '2023-10-05 10:55:12 UTC', 'event_type': 'cart', 'product_id': '5856183', 'category_id': '1487580010645553231', 'brand': 'runail', 'price': '6.27', 'user_id': '581705264', 'user_session': '1e7bf725-6c18-4c77-a21c-881380bf630b'}
{'event_time': '2023-10-05 10:55:12 UTC', 'event_type': 'view', 'product_id': '5853532', 'category_id': '1487580005050352469', 'brand': 'haruyama', 'price': '3.97', 'user_id': '339319337', 'user_session': '5f2fb254-3b61-47d1-8c00-f3a3ff2a792c'}


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 